## 📥 Step 1: Clone/Update Repository

In [ ]:
import os

repo_url = "https://github.com/Biobay/DeepLearningHard_ISWM"
branch = "main"
repo_name = repo_url.split('/')[-1]

print(f"📥 Clonazione/aggiornamento repository: {repo_url}")

# Check if we're already in the repo directory
current_dir = os.path.basename(os.getcwd())
if current_dir == repo_name:
    print(f"✅ Già nella directory '{repo_name}'. Aggiornamento...")
    !git fetch
    !git pull origin {branch}
    !git checkout {branch}
elif os.path.exists(repo_name):
    print(f"📁 Directory '{repo_name}' esiste. Aggiornamento repository...")
    os.chdir(repo_name)
    !git fetch
    !git pull origin {branch}
    !git checkout {branch}
else:
    print("📦 Clonazione repository...")
    !git clone -b {branch} {repo_url}
    os.chdir(repo_name)

print(f"\n✅ Working directory: {os.getcwd()}")

## 📦 Step 2: Installa Dipendenze

In [ ]:
print("📦 Installazione dipendenze...")

# Installa PyTorch e dipendenze
!pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cu121
!pip install -q opencv-python matplotlib Pillow tqdm tensorboard numpy kaggle

# Verifica installazione
import torch
import sys

print(f"\n✅ Python: {sys.version.split()[0]}")
print(f"✅ PyTorch: {torch.__version__}")
print(f"✅ CUDA disponibile: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️  Nessuna GPU disponibile - training sarà lento!")

## 🔑 Step 3: Configura Kaggle e Scarica Dataset

In [ ]:
# ⚠️ SOSTITUISCI CON LE TUE CREDENZIALI KAGGLE!
KAGGLE_USERNAME = "mariomastrulli"  # ← Cambia qui
KAGGLE_KEY = "KGAT_08037a2cf26b2f7ffa2612c5b6764b04"  # ← Cambia qui

import json
from pathlib import Path

# Setup credenziali Kaggle
kaggle_dir = Path.home() / ".kaggle"
kaggle_dir.mkdir(exist_ok=True)

kaggle_config = {
    "username": KAGGLE_USERNAME,
    "key": KAGGLE_KEY
}

kaggle_file = kaggle_dir / "kaggle.json"
with open(kaggle_file, 'w') as f:
    json.dump(kaggle_config, f)

os.chmod(kaggle_file, 0o600)

print("✅ Credenziali Kaggle configurate!")

# Scarica dataset
import zipfile
import shutil

KAGGLE_DATASET = "lakshaymiddha/crack-segmentation-dataset"
dataset_path = Path("dataset")
train_images = dataset_path / "train" / "images"

# Check se già esiste
if train_images.exists() and list(train_images.glob("*.jpg")):
    print(f"\n✅ Dataset già presente ({len(list(train_images.glob('*.jpg')))} immagini)")
else:
    print("\n📥 Download dataset da Kaggle (2.1 GB - può richiedere 5-10 minuti)...")
    !kaggle datasets download -d {KAGGLE_DATASET}
    
    # Estrai zip
    zip_files = list(Path(".").glob("*.zip"))
    if zip_files:
        zip_file = zip_files[0]
        print(f"📦 Estrazione {zip_file.name}...")
        
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(".")
        
        zip_file.unlink()
        
        # Organizza struttura directory
        print("📁 Organizzazione dataset...")
        dataset_path.mkdir(exist_ok=True)
        
        mappings = {
            'train_images': dataset_path / 'train' / 'images',
            'train_masks': dataset_path / 'train' / 'masks',
            'test_images': dataset_path / 'test' / 'images',
            'test_masks': dataset_path / 'test' / 'masks',
        }
        
        for src_name, dest_path in mappings.items():
            src_path = Path(src_name)
            if src_path.exists():
                dest_path.parent.mkdir(parents=True, exist_ok=True)
                shutil.move(str(src_path), str(dest_path))
        
        print("✅ Dataset scaricato e organizzato!")

# Verifica dataset
train_count = len(list((dataset_path / "train" / "images").glob("*.jpg")))
test_count = len(list((dataset_path / "test" / "images").glob("*.jpg")))
print(f"\n📊 Training images: {train_count}")
print(f"📊 Test images: {test_count}")

# Crea directory per output
for dir_name in ['models', 'checkpoints', 'predictions', 'runs']:
    Path(dir_name).mkdir(exist_ok=True)
    
print("\n✅ Setup completato!")

## 🚀 Step 4: TRAINING (50 Epoche)

**Questo richiederà ~2-3 ore con GPU (Tesla T4/P100)**

- Autoencoder convoluzionale
- MSE Loss per ricostruzione
- Checkpoints automatici ogni 5 epoche
- Resume automatico se interrotto

In [ ]:
print("🚀 Avvio training...")
print("=" * 60)

# Avvia training
!python train.py

print("\n✅ Training completato!")
print("📁 Modello salvato in: models/best_autoencoder.pth")

## 🔮 Step 5: INFERENCE

Genera maschere di predizione per tutte le immagini test

In [ ]:
print("🔮 Avvio inference...")
print("=" * 60)

!python inference.py

print("\n✅ Inference completata!")
print("📁 Predizioni salvate in: predictions/")

## 📊 Step 6: EVALUATION

Calcola metriche IoU, Dice, Precision, Recall e ottimizza threshold

In [ ]:
print("📊 Avvio evaluation...")
print("=" * 60)

!python evaluate.py

print("\n✅ Evaluation completata!")
print("📁 Risultati salvati in: results_visualization.png, threshold_optimization.png")

## ✅ COMPLETATO!

### 📁 Risultati disponibili in:

- **Modello**: `models/best_autoencoder.pth`
- **Predizioni**: `predictions/*.jpg`
- **Visualizzazioni**: `results_visualization.png`, `threshold_optimization.png`
- **Checkpoints**: `checkpoints/` (per resume)
- **TensorBoard logs**: `runs/`

### 💾 Download risultati:

```python
# Su Google Colab:
from google.colab import files
files.download('models/best_autoencoder.pth')

# Su Kaggle:
# I file saranno disponibili nell'output del notebook
```

In [ ]:
# Summary finale
print("=" * 60)
print("📊 SUMMARY FINALE")
print("=" * 60)

# Model size
model_path = Path("models/best_autoencoder.pth")
if model_path.exists():
    size_mb = model_path.stat().st_size / 1024 / 1024
    print(f"\n✅ Modello: {size_mb:.2f} MB")
else:
    print("\n⚠️  Modello non trovato")

# Predictions count
predictions = list(Path("predictions").glob("*.jpg"))
print(f"✅ Predizioni generate: {len(predictions)}")

# Visualizations
viz_files = [f for f in Path(".").glob("*.png") if "visualization" in f.name or "threshold" in f.name]
print(f"✅ Visualizzazioni: {len(viz_files)}")
for viz in viz_files:
    print(f"   - {viz.name}")

print("\n" + "=" * 60)
print("🎉 TUTTO COMPLETATO!")
print("=" * 60)